# Generating Irony data

Results for *"Qwen/Qwen2.5-0.5B-Instruct"* and *"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"*:
- irony seems a too difficult topic for this models, the "irony" sentences aren't ironic (also the "targeted" prompts case).
- doesn't follow always the instruction and is very sensitive to prompt, but it follows nicely the JSON format.

In [1]:
import os

# Move up one directory
if os.path.basename(os.getcwd()) == "mycode":
    os.chdir("..")
    
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import re
import json
from mycode.utilities import log_synthetic_data, response2json, get_response, set_seed

set_seed(42)

# file where the logs will be saved
log_file_path = "synthetic_data/semevalirony_log.json"
if os.path.exists(log_file_path):
    os.remove(log_file_path)

folder_name = "synthetic_data"
os.makedirs(folder_name, exist_ok=True) # Create folder if it doesn't exist

df = pd.read_csv("LREC-COLING/train/semevalironytrainAll.csv")
df = df.rename(columns={"1": "text", "2": "label"})
display(df.head())
first_irony = df[df['label'] == 'irony'].iloc[0].loc['text']
first_non_irony = df[df['label'] == 'non irony'].iloc[0].loc['text']

print("\nFirst irony example:")
print(first_irony)

print("\nFirst non irony example:")
print(first_non_irony)

,0,text,label
0,0,seeing ppl walking w/ crutches makes me really...,irony
1,1,"look for the girl with the broken smile, ask h...",non irony
2,2,Now I remember why I buy books online @user #s...,irony
3,3,@user @user So is he banded from wearing the c...,irony
4,4,Just found out there are Etch A Sketch apps. ...,irony



First irony example:
seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life

First non irony example:
look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵


## Qwen/Qwen2.5-0.5B-Instruct
### Baseline

In [2]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda",
    attn_implementation='flash_attention_2',
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# for qwen we use this system prompt:
system_prompt_qwen = "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."

In [3]:
prompt ="""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce 5 examples  for "irony" and 5 examples for "non irony". 

Use this format for the examples:
text: <text>
label: <label>\
"""

generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 0.9
GENERATED RESPONSE:
I'm ready to assist you with your request! Please provide me with the text and labels for "irony" and "non-irony", and I'll create the examples accordingly.


In [4]:
# With example for the two classes
prompt =f"""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce exactly 5 examples  for "irony" and 5 examples for "non irony". 

Use this format for generating the data:
text: {first_irony}
label: irony

text: {first_non_irony}
label: non irony\
"""

generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 2.78
GENERATED RESPONSE:
Sure! Here are 5 examples of ironic writing:

### Irony Example 1: 
**Text:** The cat chased the mouse around the room.
**Label:** Irony

### Irony Example 2: 
**Text:** I'm so tired, but I can't go to bed now because I have work to do.
**Label:** Irony

### Irony Example 3: 
**Text:** She gave me a book that was full of information about the benefits of exercise.
**Label:** Irony

### Irony Example 4: 
**Text:** He said he would never give up on his dream, but after years of hard work, it seems like it might not be possible.
**Label:** Irony

### Irony Example 5: 
**Text:** She wore a dress that made her feel like a princess, but when she got dressed, she realized it wasn't what she wanted.
**Label:** Irony


In [5]:
# act as a linguist and NLP practitioner
prompt =f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model

Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce exactly 5 examples for "irony" and 5 examples for "non irony". 

Use this format for generating the data:
text: <text>
label: <label>\
"""

generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 3.83
GENERATED RESPONSE:
Sure! I'll create 10 examples for each category (irony vs. non-irony) using the provided format:

### Irony Examples:
1. The teacher said, "He won't pass the exam."
2. She replied, "I'm not sure what you mean."
3. He explained, "I've never been to that place before."
4. She said, "I have no idea how to do it."
5. They laughed, "Why don't we go out for a walk?"
6. He said, "I need to get some rest."
7. She said, "It's too late already."
8. They hugged, "We're so sorry about this."
9. He said, "I really appreciate your help."
10. She said, "That sounds like fun."

### Non-Irony Examples:
1. The sky was blue.
2. The sun was shining.
3. The birds were singing.
4. The flowers were blooming.
5. The trees were tall.
6. The grass was green.
7. The water was calm.
8. The moon was bright.
9. The clouds were white.
10. The stars were shining.

Feel free to use these examples as training data for your irony detection model!


In [6]:
prompt = f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate **10 high-quality examples** of "ironic" and non-ironic statements, \
with **5 ironic** and **5 non-ironic** examples across different contexts.

### **Output Format (JSON)**
Return **only** a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
```\
"""

response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 5.14
GENERATED RESPONSE:
```json
[
    {
        "text": "I'm so happy that I got a promotion this year. It's like nothing can make me happier than seeing my boss smile at me.",
        "label": "ironic"
    },
    {
        "text": "The new kid in town just started working here. He's always smiling and making people laugh. 😃",
        "label": "non-ironic"
    },
    {
        "text": "My best friend has moved away for college. I miss him so much because he always brings laughter and energy into our lives.",
        "label": "ironic"
    },
    {
        "text": "The weather is perfect today. I'll go out and get some fresh air, because it feels like I've been sitting on a couch all day.",
        "label": "non-ironic"
    },
    {
        "text": "The old man was sick last week, but he's feeling better now. He says he needs to do more physical activity.",
        "label": "ironic"
    },
    {
        "text": "I'm going on a trip to Japan next month. I want to see the cher

In [7]:
synthetic_data = response2json(response)

print("synthetic_data[:2]: ", synthetic_data[:2])

synthetic_data[:2]:  [{'text': "I'm so happy that I got a promotion this year. It's like nothing can make me happier than seeing my boss smile at me.", 'label': 'ironic'}, {'text': "The new kid in town just started working here. He's always smiling and making people laugh. 😃", 'label': 'non-ironic'}]


In [8]:
log_synthetic_data(model_name, "baseline", prompt, synthetic_data, delta_t, output_file=log_file_path)

# Load JSON file
with open(log_file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Print the first log entry
print(json.dumps(data[:1], indent=4, ensure_ascii=False))  # Pretty-print the first entry

Logged 9 examples to synthetic_data/semevalirony_log.json. Time taken: 5.14 seconds
[
    {
        "timestamp": "2025-03-07T15:09:57.419416",
        "model": "Qwen/Qwen2.5-0.5B-Instruct",
        "generation_method": "baseline",
        "prompt": "You are an expert linguist and NLP practitioner specializing in irony detection. Your task is to generate **10 high-quality examples** of \"ironic\" and non-ironic statements, with **5 ironic** and **5 non-ironic** examples across different contexts.\n\n### **Output Format (JSON)**\nReturn **only** a valid JSON list in the following structure:\n\n```json\n[\n    {\"text\": \"seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life\", \"label\": \"ironic\"},\n    {\"text\": \"look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵\", \"label\": \"non-ironic\"},\n    ...\n]\n```",
        "time_taken_seconds": 5.14,
        "num_examples": 9,
        "generated_examp

In [9]:
# Convert log entries to a DataFrame
df = pd.DataFrame(data)

# Expand the 'generated_examples' column
samples_df = df.explode("generated_examples").reset_index(drop=True)

# Convert 'generated_examples' (which is still a dictionary) into separate columns
samples_df = pd.concat([samples_df.drop(columns=["generated_examples"]), samples_df["generated_examples"].apply(pd.Series)], axis=1)

display(samples_df.head())

,timestamp,model,generation_method,prompt,time_taken_seconds,num_examples,text,label
0,2025-03-07T15:09:57.419416,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.14,9,I'm so happy that I got a promotion this year....,ironic
1,2025-03-07T15:09:57.419416,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.14,9,The new kid in town just started working here....,non-ironic
2,2025-03-07T15:09:57.419416,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.14,9,My best friend has moved away for college. I m...,ironic
3,2025-03-07T15:09:57.419416,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.14,9,The weather is perfect today. I'll go out and ...,non-ironic
4,2025-03-07T15:09:57.419416,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.14,9,"The old man was sick last week, but he's feeli...",ironic


### Targeted synthetic data

In [10]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
List all key linguistic and semantic phenomena that must be covered by a irony detection model to perform accurately. \
Provide concrete examples illustrating each phenomenon. \
"""

response, delta_t_0 = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 8.54
GENERATED RESPONSE:
Certainly! To create an effective irony detection model, several key linguistic and semantic phenomena need to be captured:

1. **Syntax**: The structure of sentences is crucial in determining the meaning behind irony. For instance, if a sentence "She gave me a gold watch" implies that she had given her husband a gold watch, but actually meant she had just given her friend a gold watch, it would not be an ironic statement.

2. **Semantic Content**: Understanding the deeper meaning behind an expression or situation can help distinguish between literal and ironic statements. For example, saying "He was very tired" about someone being late for work could be considered a mild exaggeration because the person might have been exhausted from working hard rather than feeling sleepy.

3. **Contextual Clues**: The surrounding context plays a significant role in interpreting irony. If someone says something sarcastically in response to another's compliment, thi

In [11]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
List all key linguistic and semantic phenomena that must be considered for an irony detection model to perform accurately. \
Provide only the names of these phenomena as a structured list, without explanations. \
For example: sarcasm, negation, pragmatic inference, unexpected contrast. \
Return the list in a simple bullet-point format.\
"""

response, delta_t_0 = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 0.49
GENERATED RESPONSE:
- Irony Detection
  - Contrast
  - Subversion
  - Satire
  - Antithesis
  - Sarcasm
  - Negation


In [12]:
# Replace "- " at the beginning of each line with a comma
csv_string = re.sub(r"^\s*-\s*", "", response, flags=re.MULTILINE)  # Remove bullet points
csv_string = ", ".join(csv_string.strip().split("\n"))  # Join lines with ", "

prompt = f"""\
Generate 10 realistic sentences illustrating irony detection examples involving {csv_string}. Ensure that:
- 5 sentences are **ironic**.
- 5 sentences are **non-ironic**.

### **Output Format (JSON)**
Return **only** a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
```\
"""

response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)
delta_t += delta_t_0 # Add the time taken for the previous response

# add to log
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 4.21
GENERATED RESPONSE:
```json
[
    {
        "text": "the sun is shining brightly today, making everyone's day brighter",
        "label": "ironic"
    },
    {
        "text": "the sky is blue, but it's raining hard outside",
        "label": "ironic"
    },
    {
        "text": "the moon is full, but there's no one around to watch",
        "label": "ironic"
    },
    {
        "text": "the wind blows gently, but it's cold and blowing from the wrong direction",
        "label": "ironic"
    },
    {
        "text": "the flowers are blooming, but they're not beautiful",
        "label": "ironic"
    },
    {
        "text": "the leaves are changing color, but they're still green",
        "label": "ironic"
    },
    {
        "text": "the birds are singing, but they're not happy",
        "label": "ironic"
    },
    {
        "text": "the water is crystal clear, but it's not refreshing",
        "label": "ironic"
    },
    {
        "text": "the clouds are gray, b

---

### deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B

In [13]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation='flash_attention_2',
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.generation_config.pad_token_id = tokenizer.pad_token_id

# we don't submit a system prompt as suggested in the model card

In [14]:
prompt ="""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce 5 examples  for "irony" and 5 examples for "non irony". 

Use this format for the examples:
text: <text>
label: <label>\
"""

generated, delta_t = get_response(prompt, model, tokenizer, max_new_tokens=2048) # increase max_new_tokens to 1024 for deepseek

TIME TAKEN: 31.71
GENERATED RESPONSE:
Okay, so I need to come up with 10 examples of irony in written text, dividing them into 5 that are categorized as "irony" and 5 that are "non irony." The user provided a specific format where each example has a text and a label. I should make sure each text clearly shows the irony and label appropriately.

First, I should understand what irony is. It's a situation where the intended meaning of two statements is opposite of each other. So, one statement is true and the other is false, or vice versa.

For the "irony" category, I need to find examples where the irony is more pronounced, maybe with stronger statements or more complex situations. For "non irony," I should look for statements where the meaning is clear and the truth is straightforward.

Let me brainstorm some text examples.

1. "The sky is blue, but the ground is red." - That's a classic irony because the sky is actually blue, but the ground is red in winter. So, the label should be "ir

In [15]:
# With example for the two classes
prompt =f"""\
Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model. \
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce exactly 5 examples  for "irony" and 5 examples for "non irony". 

Use this format for generating the data:
text: {first_irony}
label: irony

text: {first_non_irony}
label: non irony\
"""

generated, delta_t = get_response(prompt, model, tokenizer, max_new_tokens=2048)

TIME TAKEN: 33.32
GENERATED RESPONSE:
Okay, so I need to generate 10 synthetic examples for an irony detection model. The user wants exactly 5 irony examples and 5 non-ironic ones. Let me think about how to approach this.

First, I should understand what makes text irony. Irony occurs when there's a contradiction or imbalance in meaning, often involving words that have opposite meanings but are used in a way that creates a sense of imbalance or surprise. 

For the irony examples, I should create situations where the words used suggest an opposite meaning. Maybe using words like "crutches," "broken smile," "broken word," "broken promise," "broken promise," "broken promise," "broken promise," "broken promise," and "broken promise." These words are all negative and could imply a lack of hope or a negative outcome.

For non-ironic examples, I need to avoid such contradictions. Maybe use positive language or situations where the meanings align. Words like "looking for the girl with the brok

In [16]:
# act as a linguist and NLP practitioner
prompt =f"""\
<context>
You are an expert linguist and NLP practitioner specializing in irony detection.
</context>

Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model. \
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce exactly 5 examples for "irony" and 5 examples for "non irony". 

Use this format for generating the data:
text: <text>
label: <label>\
"""

generated, delta_t = get_response(prompt, model, tokenizer, max_new_tokens=2048)

TIME TAKEN: 35.45
GENERATED RESPONSE:
Alright, I'm trying to figure out how to approach this request. The user wants me to generate synthetic examples for irony detection. They specifically asked for 10 examples, with exactly 5 being irony and 5 being non-ironic. Each example should be in a specific format: text and a label.

First, I need to understand what constitutes irony. Irony is when the meaning of something is opposite to what is implied. It often involves a pun, contradiction, or inverted logic. For instance, "Why did the chicken cross the road?" is ironic because the answer is a joke, not a question.

I should start by brainstorming 5 ironic sentences. Maybe include some common idioms and phrases that are known for their irony. For example, "What do you want of me?" is a classic one because it's a double negative. Another could be "Why did the scarecrow win an award?" which is a play on "Why did the chicken cross the road?" since both are about unexpected success.

Next, I'll

In [17]:
prompt = f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate **10 high-quality examples** of ironic and non-ironic statements, \
with **5 ironic** and **5 non-ironic** examples across different contexts.

### **Output Format (JSON)**
Return the final result into a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
```\
"""

response, delta_t = get_response(prompt, model, tokenizer, max_new_tokens=2048)

# add to log
synthetic_data = response2json(response)
log_synthetic_data(model_name, "baseline", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 17.05
GENERATED RESPONSE:
Okay, so I have this task where I need to generate 10 examples of ironic and non-ironic statements. The user wants 5 each, but they also specified a specific output format. Let me break this down.

First, I need to understand what irony is. Irony is a form of figurative expression where the meaning of a statement is contrary to its literal meaning. It often involves a clever twist or surprise, making the statement seem surprising or amusing.

For the ironic examples, I should think of statements that have a twist, maybe involving situations where expectations aren't met. Maybe something like a situation where the outcome is contrary to what is expected, or a situation where a simple action leads to unexpected consequences.

I should consider different contexts. Maybe natural phenomena, everyday situations, or even more abstract concepts. For example, someone wearing a hat when they're actually going to the gym, or someone giving advice that seems c

### Targeted synthetic data


In [18]:
prompt = f"""\
You are an expert linguistics and NLP practitioner. \
List all key linguistic and semantic phenomena that must be considered for an irony detection model to perform accurately. \
Provide only the names of these key concepts as a structured list, without explanations.

Return the list in a simple bullet-point format, each bullet point must start with a dash "-".\
"""

response, delta_t_0 = get_response(prompt, model, tokenizer, max_new_tokens=2048)

# Extract bullet points
bullets = re.findall(r"- (.+)", response)
bullets = [b.strip() for b in bullets]  # Remove leading/trailing whites

# Convert to CSV string
csv_string = ", ".join(bullets)

TIME TAKEN: 8.48
GENERATED RESPONSE:
Okay, so the user is asking for a list of key linguistic and semantic phenomena that an irony detection model needs to consider. They want this structured as bullet points starting with a dash. Hmm, I need to make sure I cover everything important.

First, I should think about the different types of irony. There's the direct form, where the meaning is explicitly stated, like "I'm so sorry, but I can't believe you're right." Then there's the indirect form, where it's not immediately obvious, like "I told you, but you're not believing me."

Next, the user mentioned the "irony triangle." That's a common one, involving a premise, conclusion, and a premise that contradicts the conclusion. It's a classic way to detect irony.

Also, the user might be concerned about context. Without context, it's hard to determine if something is irony. So, contextual factors are important.

I should include how the model handles sarcasm. sarcasm often uses double negative

In [19]:
prompt = f"""\
Generate 10 realistic sentences illustrating irony detection examples involving {csv_string}. Ensure that:
- 5 sentences are **ironic**.
- 5 sentences are **non-ironic**.

### **Output Format (JSON)**
Return **only** a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
```\
"""

response, delta_t = get_response(prompt, model, tokenizer, max_new_tokens=2048)
delta_t += delta_t_0 # Add the time taken for the previous response

# add to log
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 20.22
GENERATED RESPONSE:
Okay, so the user wants me to generate 10 sentences that illustrate different types of irony. They specified 5 should be ironic and 5 non-ironic, and each sentence needs a label. The sentences should cover various types like Direct Irony, Indirect Irony, Irony Triangle, Sarcasm, Context, Ambiguity, Multi-Level Irony, Strong Irony, Rhetorical Question, and Irony in Context.

First, I need to make sure I understand each type of irony. Direct Irony is when the meaning of the sentence is directly opposed by the word choice. Indirect Irony involves the contradiction being hidden within a larger context. Irony Triangle is when the contradiction is between the speaker and the listener. Sarcasm is when the meaning is meant to be critical but is expressed in a light way. Context is when the situation affects the meaning. Ambiguity is when the meaning isn't clear because of word choice. Multi-Level Irony is when there are multiple layers of irony. Strong Iro